In [ ]:
# | default_exp batch_job_components.fastapi

In [ ]:
from airt.testing import activate_by_import

[INFO] airt.testing.activate_by_import: Testing environment activated.
[INFO] numexpr.utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO] numexpr.utils: NumExpr defaulting to 8 threads.
[INFO] airt.keras.helpers: Using a single GPU #0 with memory_limit 1024 MB


In [ ]:
# | export

from typing import *

import airt_service.sanitizer
from airt.logger import get_logger
from airt_service.background_task import execute_cli
from airt_service.batch_job_components.base import BatchJobContext

In [ ]:
from airt_service.helpers import set_env_variable_context
from fastapi import BackgroundTasks

In [ ]:
# | exporti

logger = get_logger(__name__)

In [ ]:
# | export


class FastAPIBatchJobContext(BatchJobContext):
    def __init__(self, task: str, **kwargs):
        """FastAPI Batch Job Context

        Do not use __init__, please use factory method `create` to initiate object
        """
        BatchJobContext.__init__(self, task=task)
        self.background_tasks = kwargs["background_tasks"]

    def create_job(self, command: str, environment_vars: Dict[str, str]):
        """Create a new job

        Args:
            command: Command to execute in job
            environment_vars: Environment vars to set in the container
        """
        logger.info(
            f"{self.__class__.__name__}.create_job({self=}, {command=}, {environment_vars=})"
        )
        self.background_tasks.add_task(execute_cli, command=command)


FastAPIBatchJobContext.add_factory()

In [ ]:
with set_env_variable_context(variable="JOB_EXECUTOR", value="fastapi"):
    background_tasks = BackgroundTasks()
    with BatchJobContext.create(
        "csv_processing", background_tasks=background_tasks
    ) as batch_ctx:
        display(batch_ctx)
        assert batch_ctx.__class__.__name__ == "FastAPIBatchJobContext"

        batch_ctx.create_job(command="ls -lah", environment_vars={})

        bg_task = background_tasks.tasks[-1]
        display(f"{bg_task.func=}", f"{bg_task.args=}", f"{bg_task.kwargs=}")
        assert bg_task.func == execute_cli
        assert bg_task.kwargs["command"] == f"ls -lah"

[INFO] airt_service.batch_job_components.base: Entering FastAPIBatchJobContext(task=csv_processing)


FastAPIBatchJobContext(task=csv_processing)

[INFO] __main__: FastAPIBatchJobContext.create_job(self=FastAPIBatchJobContext(task=csv_processing), command='ls -lah', environment_vars={})


'bg_task.func=<function execute_cli>'

'bg_task.args=()'

"bg_task.kwargs={'command': 'ls -lah'}"

[INFO] airt_service.batch_job_components.base: Exiting FastAPIBatchJobContext(task=csv_processing): exc_type=None, exc=None, None
